In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 15705, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 15705 (delta 9), reused 14 (delta 4), pack-reused 15672
Receiving objects: 100% (15705/15705), 14.51 MiB | 29.12 MiB/s, done.
Resolving deltas: 100% (10750/10750), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [3]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.1 MB/s eta 0:00:00
Setup complete. Using torch 2.0.1+cu118 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15101MB, multi_processor_count=40)


In [5]:
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="zkyt15pyUkXSH9tlso0C")
project = rf.workspace("noor-djz41").project("multiuav")
dataset = project.version(4).download("yolov5")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to MultiUav-4 in yolov5pytorch:: 100%|██████████| 19773/19773 [00:04<00:00, 4837.23it/s]


In [6]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [7]:
#this is the model configuration we will use for our tutorial 
%cat /content/yolov5/models/hub/yolov5s-transformer.yaml

# YOLOv5 🚀 by Ultralytics, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [1024]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'ne

In [8]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, CrossConv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3x, [128]],
   [-1, 1, CrossConv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3x, [256]],
   [-1, 1, CrossConv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3x, [512]],
   [-1, 1, CrossConv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [1024]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

In [15]:
%%writetemplate /content/yolov5/models/YOLOv5-CVIT-PANet.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32
# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3TR, [1024]],  # 9 <--- C3TR() Transformer module
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, CrossConv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3x, [512, False]],  # 13

   [-1, 1, CrossConv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3x, [256, False]],  # 17 (P3/8-small)

   [-1, 1, CrossConv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3x, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, CrossConv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3x, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]


In [16]:
%%time
%cd /content/yolov5/
##
!python train.py --img 416 --batch-size -1 --optimizer AdamW --cos-lr --label-smoothing 0.01  --patience 50  --epochs 1000 --hyp data/hyps/hyp.scratch-high.yaml --data {dataset.location}/data.yaml --cfg ./models/YOLOv5-CVIT-PANet.yaml --weights '' --project "/content/gdrive/MyDrive/Runs" --name YOLOv5-CVIT-PANet_results  --cache

/content/yolov5
2023-05-27 07:59:19.841931: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 07:59:20.759318: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=./models/YOLOv5-CVIT-PANet.yaml, data=/content/yolov5/MultiUav-4/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=1000, batch_size=-1, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=YOLOv5-CVIT-PANet_results, exist_ok=False, quad=False, cos_lr=True, label_smoothing=0.01, patience=50

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
!python train.py --img 416 --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOv5-CVIT-PANet_results  --cache  --resume "/content/gdrive/MyDrive/Runs/YOLOv5-CVIT-PANet_results/weights/last.pt"

In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason... 
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/YOLOv5-CVIT-PANet/results.png', width=1000)  # view results.png

In [ ]:
# trained weights are saved by default in our weights folder
%ls runs/
%ls runs/train/YOLOv5-CVIT-PANet_results/weights

In [ ]:
#########Validate whole dataset
# Validate model and print evaluation metrics
!python val.py --weights runs/train/YOLOv5-CVIT-PANet_results/weights/best.pt --data {dataset.location}/data.yaml --img 256 --half

In [ ]:
######## Test Valid image folder
%cd /content/yolov5/
!python detect.py --weights runs/train/YOLOv5-CVIT-PANet_results/weights/best.pt --img 320 --conf 0.25 --source /content/yolov5/MultiUav-3/valid/images

In [ ]:
!python val.py --task study --weights runs/train/YOLOv5-CVIT-PANet_results/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
!python val.py --task speed --weights runs/train/YOLOv5-CVIT-PANet_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 

In [ ]:
##I have created a zip file:
!zip -r /content/YOLOv5-CVIT.zip /content/yolov5/runs/
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/YOLOv5-CVIT.zip")